##problem
1. as a doctor I need to be able easily see where my team is on a weekly basis 
2. as a doctor with responsibility for the weekly rota I need a way of generating a weekly view from the the master rota that is robust and time efficient
n.b the current approach of manually creating a weekly view of the master rota is prone to transcription error and a waste of clinical time

##approach
1. automate the task of creating a weekly view of the master rota with python
2. longer term, maybe a giant excel spreadsheet isn't the optimal master rota solution?

In [1]:
import pandas as pd
import datetime
import collections

In [2]:
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
        return d + datetime.timedelta(days_ahead)
    
def next_week(df):
    d = datetime.date.today()
    next_monday = next_weekday(d, 0) # 0 = Monday, 1=Tuesday, 2=Wednesday...
    following_monday = next_monday + datetime.timedelta(13)
    weekly_rota = df[next_monday:following_monday] #next 2 weeks cut
    return weekly_rota

def load_master_rota():
    master_rota = 'data/wota.xlsx'

    cols = range(0,45)
    df = pd.read_excel(master_rota, dayfirst=True, skiprows=1340, usecols=cols) #load our spreadsheet
    
    current_cols = ['Date', 'Day', 'Notes', 'ConsOnCallNote1', 'ConsOnCallNote2', 
                'First Floor Consultant', 'Manvers Consultant','CXH AMT', '4S CXH', 'HH referrals',
                'MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 'GR', 'LM',
                'Date1', 'Day1', 'SprNote1', 'SprNote2', 'NoReg1', 'NoReg2', 'Chris', 'Carl', 'Simon', 'Reza', 
                'Date2', 'Day2', 'Jennifer', 'Louis', 'Chinonso', 'NoSHO1', 'NoSHO2', 'NoSHO3', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne'] #define column headings

    reg = ['Chris', 'Carl', 'Simon', 'Reza']

    assert len(df.columns) == len(current_cols) #sanity check

    df.columns = current_cols #set our column names

    df = df[['Date', 'Day', 'First Floor Consultant', 'Manvers Consultant', 
                'MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 'GR', 'LM',
                'Chris', 'Carl', 'Simon', 'Reza', 
                'Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']]  #throw away columns we don't need

    df.index = df.Date #lets index on the date because its better that way
    return df

In [3]:
def FirstFloorReg(df):
    FirstFloorReg = {}
    reg = ['Chris', 'Carl', 'Simon', 'Reza']
    
    for d in df['Date']:
        for registrar in reg:
            if df[d:d][registrar][(df[d:d][registrar].astype(str).str.contains('1'))].any():
                FirstFloorReg[d] = registrar 
            elif df[d:d][registrar][(df[d:d][registrar].astype(str).str.contains('PT'))].any():
                FirstFloorReg[d] = registrar + '(post-take)'
            elif df[d:d][registrar][(df[d:d][registrar].astype(str).str.contains('Take'))].any():
                FirstFloorReg[d] = registrar + '(on-call)'
                
    return FirstFloorReg

def FirstFloorJunior(df):
    FirstFloorJunior = {}
    juniors = ['Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']
    
    for d in df['Date']:
        for junior in juniors:
            if df[d:d][junior][(df[d:d][junior].astype(str).str.contains('1'))].any():
                if d.to_pydatetime() in FirstFloorJunior:
                    FirstFloorJunior[d.to_pydatetime()].append(junior)
                else:
                    FirstFloorJunior[d.to_pydatetime()] = [junior]
            elif df[d:d][junior][(df[d:d][junior].astype(str).str.contains('PT'))].any():
                if d.to_pydatetime() in FirstFloorJunior:
                    FirstFloorJunior[d.to_pydatetime()].append(junior + '(post-take)')
                else:
                    FirstFloorJunior[d.to_pydatetime()] = [junior]
            elif df[d:d][junior][(df[d:d][junior].astype(str).str.contains('Take'))].any():
                if d.to_pydatetime() in FirstFloorJunior:
                    FirstFloorJunior[d.to_pydatetime()].append(junior + '(on-call)')
                else:
                    FirstFloorJunior[d.to_pydatetime()] = [junior]

    return FirstFloorJunior


def SecondFloorReg(df):
    SecondFloorReg = {}
    reg = ['Chris', 'Carl', 'Simon', 'Reza']
    
    for d in df['Date']:
        for registrar in reg:
            if df[d:d][registrar][(df[d:d][registrar].astype(str).str.contains('2'))].any():
                SecondFloorReg[d] = registrar 
                
    return SecondFloorReg

def SecondFloorJunior(df):
    SecondFloorJunior = {}
    juniors = ['Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']
    
    for d in df['Date']:
        for junior in juniors:
            if df[d:d][junior][(df[d:d][junior].astype(str).str.contains('2'))].any():
                if d.to_pydatetime() in SecondFloorJunior:
                    SecondFloorJunior[d.to_pydatetime()].append(junior)
                else:
                    SecondFloorJunior[d.to_pydatetime()] = [junior]

    return SecondFloorJunior

def ClinicReg(df):
    ClinicReg = {}
    reg = ['Chris', 'Carl', 'Simon', 'Reza']
    
    for d in df['Date']:
        for registrar in reg:
            if df[d:d][registrar][(df[d:d][registrar].astype(str).str.contains('CLINIC'))].any():
                ClinicReg[d] = registrar 
                
    return ClinicReg


def Night(df):
    Night = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 'GR', 'LM',
                'Chris', 'Carl', 'Simon', 'Reza', 
                'Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str).str.contains('Night'))].any():
                if d.to_pydatetime() in Night:
                    Night[d.to_pydatetime()].append(s)
                else:
                    Night[d.to_pydatetime()] = [s]
    return Night

def ZeroDay(df):
    ZeroDay = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 'GR', 'LM',
                'Chris', 'Carl', 'Simon', 'Reza', 
                'Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str).str.contains('Zero'))].any():
                if d.to_pydatetime() in ZeroDay:
                    ZeroDay[d.to_pydatetime()].append(s)
                else:
                    ZeroDay[d.to_pydatetime()] = [s]
    return ZeroDay

def AnnualLeave(df):
    AnnualLeave = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Simon', 'Reza', 
                'Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str).str.contains('AL'))].any():
                if d.to_pydatetime() in AnnualLeave:
                    AnnualLeave[d.to_pydatetime()].append(s)
                else:
                    AnnualLeave[d.to_pydatetime()] = [s]

    return AnnualLeave

def StudyLeave(df):
    StudyLeave = {}
    staff = ['MB', 'SE', 'LYH', 'OMK', 'WO', 'MW', 'PM', 'IC', 'SB', 'CR', 
                'Chris', 'Carl', 'Simon', 'Reza', 
                'Jennifer', 'Louis', 'Chinonso', 'Sommer', 
                'Miriam', 'Ved', 'Ernest', 'Anne']
    
    for d in df['Date']:
        for s in staff:
            if df[d:d][s][(df[d:d][s].astype(str).str.contains('TRAINING DAY'))].any():
                if d.to_pydatetime() in StudyLeave:
                    StudyLeave[d.to_pydatetime()].append(s)
                else:
                    StudyLeave[d.to_pydatetime()] = [s]
            elif df[d:d][s][(df[d:d][s].astype(str).str.contains('SL'))].any():
                if d.to_pydatetime() in StudyLeave:
                    StudyLeave[d.to_pydatetime()].append(s)
                else:
                    StudyLeave[d.to_pydatetime()] = [s]
                    
    return StudyLeave

def MorningClinic(df):
    MorningClinic = {}
    for d in df['Date']:
        if d.weekday() == 0:
            MorningClinic[d] = 'ILD : no reg'
        elif d.weekday() == 1:
             MorningClinic[d] = 'TB : no reg'
        elif d.weekday() == 2:
             MorningClinic[d] = 'COPD/GEN : SHO'
        elif d.weekday() == 3:
             MorningClinic[d] = 'MDT : no reg'
        elif d.weekday() == 4:
             MorningClinic[d] = ''
    
    return MorningClinic
    
def AfternoonClinic(df):
    AfternoonClinic = {}
    for d in df['Date']:
        if d.weekday() == 0:
            AfternoonClinic[d] = 'Elkin : SHO'
        elif d.weekday() == 1:
             AfternoonClinic[d] = ''
        elif d.weekday() == 2:
             AfternoonClinic[d] = 'General : no reg'
        elif d.weekday() == 3:
             AfternoonClinic[d] = 'Fast track : no reg'
        elif d.weekday() == 4:
             AfternoonClinic[d] = 'Ross : Pleural list'
    
    return AfternoonClinic

def Bronch(df):
    Bronch = {}
    for d in df['Date']:
        if d.weekday() == 0:
            Bronch[d] = 'Han/Bloch : no reg'
        elif d.weekday() == 1:
             Bronch[d] = 'Elkin/Counihan : no reg'
        elif d.weekday() == 2:
             Bronch[d] = 'Berry/Ross : no reg'
        elif d.weekday() == 3:
             Bronch[d] = 'Wick : no reg'
        elif d.weekday() == 4:
             Bronch[d] = 'Kon : no reg'
    
    return Bronch


In [4]:
df = load_master_rota()
df = next_week(df)

In [5]:
df.T

Date,2016-10-03 00:00:00,2016-10-04 00:00:00,NaT,2016-10-05 00:00:00,2016-10-06 00:00:00,2016-10-07 00:00:00,2016-10-08 00:00:00,2016-10-09 00:00:00,2016-10-10 00:00:00,2016-10-11 00:00:00,2016-10-12 00:00:00,2016-10-13 00:00:00,2016-10-14 00:00:00,2016-10-15 00:00:00,2016-10-16 00:00:00
Date,2016-10-03 00:00:00,2016-10-04 00:00:00,NaN,2016-10-05 00:00:00,2016-10-06 00:00:00,2016-10-07 00:00:00,2016-10-08 00:00:00,2016-10-09 00:00:00,2016-10-10 00:00:00,2016-10-11 00:00:00,2016-10-12 00:00:00,2016-10-13 00:00:00,2016-10-14 00:00:00,2016-10-15 00:00:00,2016-10-16 00:00:00
Day,Monday,Tuesday,NaN,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
First Floor Consultant,SB,SB,NaN,SB,SB,SB,SB,SB,SB,SB,CR,CR,CR,NaN,NaN
Manvers Consultant,WARDS: PM,WARDS: PM,NaN,WARDS: PM,WARDS: PM,WARDS: PM,NaN,NaN,WARDS: PM,WARDS: PM,WARDS: PM,WARDS: PM,WARDS: PM,NaN,NaN
MB,NaN,NaN,NaN,NaN,NaN,sl,NaN,NaN,AL,NaN,AL,AL,NaN,NaN,NaN
SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LYH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OMK,12,13,NaN,14,15,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
first = (('First Floor Registrar (#1028)', FirstFloorReg(df)), 
         ('First Floor Juniors (#1656, 1016)', FirstFloorJunior(df)))

second = (('Second Floor Registrar (#1238)', SecondFloorReg(df)),
         ('Second Floor Juniors (#1208, 1138)', SecondFloorJunior(df)))

other = (('Morning Clinic : Reg', MorningClinic(df)),
        ('Afternoon Clinic : Reg', AfternoonClinic(df)),
        ('Bronchoscopy : Reg', Bronch(df)),
        ('Clinic Registrar', ClinicReg(df)),
        ('Nights', Night(df)),
        ('Zero Days', ZeroDay(df)),
        ('Annual Leave', AnnualLeave(df)),
        ('Study Leave', StudyLeave(df)))

In [7]:
first = collections.OrderedDict(first)
second = collections.OrderedDict(second)
other = collections.OrderedDict(other)

a = df.ix[:,1:3].transpose()
b = pd.DataFrame(first).transpose()
c = df.ix[:,3:4].transpose()
d = pd.DataFrame(second).transpose()
e = pd.DataFrame(other).transpose()

data_frames = [a,b,c,d,e]

weeklyrota = pd.concat(data_frames).fillna('')

In [8]:
weeklyrota

,NaT,2016-10-03 00:00:00,2016-10-04 00:00:00,2016-10-05 00:00:00,2016-10-06 00:00:00,2016-10-07 00:00:00,2016-10-08 00:00:00,2016-10-09 00:00:00,2016-10-10 00:00:00,2016-10-11 00:00:00,2016-10-12 00:00:00,2016-10-13 00:00:00,2016-10-14 00:00:00,2016-10-15 00:00:00,2016-10-16 00:00:00
Day,,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
First Floor Consultant,,SB,SB,SB,SB,SB,SB,SB,SB,SB,CR,CR,CR,,
First Floor Registrar (#1028),,,,,,,Chris(on-call),Chris(post-take),Carl,Carl,Carl(on-call),Carl(post-take),Chris,,
"First Floor Juniors (#1656, 1016)",,[Chinonso],"[Jennifer, Chinonso, Sommer(post-take)]","[Jennifer, Chinonso, Sommer]","[Jennifer, Miriam(on-call)]","[Jennifer, Chinonso, Sommer, Miriam(post-take)]",[Chinonso],[Chinonso],"[Jennifer, Ernest]","[Jennifer, Ernest]","[Louis, Ernest]","[Jennifer, Louis(post-take), Ernest]","[Louis, Ved]",,
Manvers Consultant,,WARDS: PM,WARDS: PM,WARDS: PM,WARDS: PM,WARDS: PM,,,WARDS: PM,WARDS: PM,WARDS: PM,WARDS: PM,WARDS: PM,,
Second Floor Registrar (#1238),,,,,,,,,Simon,Simon,Simon,,Simon,,
"Second Floor Juniors (#1208, 1138)",,"[Sommer, Ernest]",[Ernest],,,[Ved],[Ved],[Ved],"[Sommer, Miriam]","[Sommer, Miriam, Ved]","[Jennifer, Sommer, Miriam, Ved]","[Chinonso, Sommer, Miriam, Ved]","[Chinonso, Miriam]",,
Morning Clinic : Reg,,ILD : no reg,TB : no reg,COPD/GEN : SHO,MDT : no reg,,,,ILD : no reg,TB : no reg,COPD/GEN : SHO,MDT : no reg,,,
Afternoon Clinic : Reg,,Elkin : SHO,,General : no reg,Fast track : no reg,Ross : Pleural list,,,Elkin : SHO,,General : no reg,Fast track : no reg,Ross : Pleural list,,
Bronchoscopy : Reg,,Han/Bloch : no reg,Elkin/Counihan : no reg,Berry/Ross : no reg,Wick : no reg,Kon : no reg,,,Han/Bloch : no reg,Elkin/Counihan : no reg,Berry/Ross : no reg,Wick : no reg,Kon : no reg,,


In [9]:
d = datetime.date.today()
week_starts = next_weekday(d, 0)

In [10]:
rotaname = '%s_weekly_rota.csv' % next_weekday(d, 0)
rotaname2 = '%s_weekly_rota.xls' % next_weekday(d, 0)

In [11]:
weeklyrota.to_csv(rotaname)
weeklyrota.to_excel(rotaname2)

In [12]:
df['Reza']

Date
2016-10-03          AL
2016-10-04         NaN
NaT                NaN
2016-10-05    Standard
2016-10-06    Standard
2016-10-07    Standard
2016-10-08         NaN
2016-10-09         NaN
2016-10-10      CLINIC
2016-10-11      CLINIC
2016-10-12      CLINIC
2016-10-13          SL
2016-10-14      CLINIC
2016-10-15         NaN
2016-10-16         NaN
Name: Reza, dtype: object

In [13]:
df.head()

,Date,Day,First Floor Consultant,Manvers Consultant,MB,SE,LYH,OMK,WO,MW,...,Simon,Reza,Jennifer,Louis,Chinonso,Sommer,Miriam,Ved,Ernest,Anne
Date,,,,,,,,,,,,,,,,,,,,,
2016-10-03,2016-10-03,Monday,SB,WARDS: PM,NaN,NaN,NaN,12,NaN,NaN,...,Zero,AL,Zero,Night,0900-1730,2nd on,AL,Zero,2nd on,Zero
2016-10-04,2016-10-04,Tuesday,SB,WARDS: PM,NaN,NaN,NaN,13,NaN,NaN,...,Standard,NaN,0900-1730,Night,0900-1730,PT,AL,AL,2nd on,Standard
NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-05,2016-10-05,Wednesday,SB,WARDS: PM,NaN,NaN,NaN,14,NaN,NaN,...,Standard,Standard,0900-1730,Night,0900-1730,0900-1730,Standard,Standard,Standard,Standard
2016-10-06,2016-10-06,Thursday,SB,WARDS: PM,NaN,NaN,NaN,15,NaN,NaN,...,Standard,Standard,0900-1730,Night,SL,SL,Take,Standard,Standard,SL


In [14]:
df['Simon']

Date
2016-10-03        Zero
2016-10-04    Standard
NaT                NaN
2016-10-05    Standard
2016-10-06    Standard
2016-10-07    Standard
2016-10-08         NaN
2016-10-09         NaN
2016-10-10           2
2016-10-11           2
2016-10-12           2
2016-10-13          SL
2016-10-14           2
2016-10-15         NaN
2016-10-16         NaN
Name: Simon, dtype: object